In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [45]:
data = pd.read_csv("./pison_data_interview.csv", header=None, names=["timestamp","chan0_raw","chan1_raw","chan0_hp","chan1_hp","quat_x","quat_y","quat_z","quat_w","gyro_x","gyro_y","gyro_z","acc_x","acc_y","acc_z","label","rep"])


In [46]:
data

,timestamp,chan0_raw,chan1_raw,chan0_hp,chan1_hp,quat_x,quat_y,quat_z,quat_w,gyro_x,gyro_y,gyro_z,acc_x,acc_y,acc_z,label,rep
0,1514824.503,12535249,12566283,-11889,17295,0.321960,-0.596619,-0.621826,0.392090,-1.342870,1.063105,0.503576,-0.660156,-10.003906,1.21875,0,1
1,1514827.496,12536264,12559246,-2063,13384,0.321960,-0.596741,-0.621826,0.392029,-1.510729,1.175011,0.000000,-0.660156,-10.003906,1.21875,0,1
2,1514830.493,12538584,12565279,2757,16008,0.321960,-0.596741,-0.621765,0.391968,-1.230964,1.175011,-0.279765,-0.660156,-10.003906,1.21875,0,1
3,1514833.500,12546745,12567024,7504,5644,0.321960,-0.596741,-0.621765,0.391968,-1.230964,1.175011,-0.279765,-0.660156,-10.003906,1.21875,0,1
4,1514836.498,12537375,12545467,-3855,-15893,0.321960,-0.596802,-0.621765,0.391907,-1.063105,0.839294,-0.727388,-0.660156,-10.003906,1.21875,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14976,1618167.393,11942866,12916517,-32480,-78200,-0.312927,-0.112427,-0.932861,0.138855,-114.255844,-1219.997200,82.026960,0.363281,-1.464844,-9.00000,4,3
14977,1618170.388,11881160,12934589,-43544,-69885,-0.312927,-0.112427,-0.932861,0.138855,-114.255844,-1219.997200,82.026960,0.363281,-1.464844,-9.00000,4,3
14978,1618173.377,11869705,12927326,-25592,-73242,-0.264465,-0.108093,-0.946472,0.150146,-104.464080,-1181.054100,103.904560,0.363281,-1.464844,-9.00000,4,3
14979,1618176.442,11898127,12957446,40027,-17502,-0.218140,-0.103271,-0.956909,0.161560,-94.336610,-1117.715300,125.782140,0.363281,-1.464844,-9.00000,4,3


In [47]:
features = ["chan0_hp", "chan1_hp", "gyro_x", "gyro_y", "gyro_z", "acc_x", "acc_y", "acc_z"]


In [48]:
# create function to return trial number

def create_trial_vals(row):
    if row["label"] == 0 and row["rep"] == 1:
        return 1
    elif row["label"] == 1 and row["rep"] == 1:
        return 2
    elif row["label"] == 2 and row["rep"] == 1:
        return 3
    elif row["label"] == 3 and row["rep"] == 1:
        return 4
    elif row["label"] == 4 and row["rep"] == 1:
        return 5
    elif row["label"] == 0 and row["rep"] == 2:
        return 6
    elif row["label"] == 1 and row["rep"] == 2:
        return 7
    elif row["label"] == 2 and row["rep"] == 2:
        return 8
    elif row["label"] == 3 and row["rep"] == 2:
        return 9
    elif row["label"] == 4 and row["rep"] == 2:
        return 10
    elif row["label"] == 0 and row["rep"] == 3:
        return 11
    elif row["label"] == 1 and row["rep"] == 3:
        return 12
    elif row["label"] == 2 and row["rep"] == 3:
        return 13
    elif row["label"] == 3 and row["rep"] == 3:
        return 14
    elif row["label"] == 4 and row["rep"] == 3:
        return 15

In [49]:
data["trial"] = data.apply(lambda row: create_trial_vals(row), axis=1)

In [50]:
data_1 = data[(data.trial == 1)]

In [51]:
test_string = ''

for feature in features:
    for trial in range(1,16):
        for index, item in data[(data[[features]].trial == trial)].items():
            if item != 0:
                test_string = test_string + f'{index}:{item} '
            
        test_string = test_string[:-1]


9843


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]